# 1. Import Packages and Define Functions

In [79]:
# Import some libraries that will be used
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import mysql.connector
import sys
sys.path.insert(1, '/Users/brianmccabe/DataScience/Flatiron/mod5/Emoji_Analysis/Scripts/')
import config
import warnings
warnings.filterwarnings('ignore')



pd.set_option('display.max_columns', 300)

%matplotlib inline

In [80]:
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from nltk.probability import FreqDist
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn import metrics
from sklearn.model_selection import train_test_split
from matplotlib import cm
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.naive_bayes import MultinomialNB
import string
import scipy
import emoji
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import re
from textblob import TextBlob
seed=42
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/brianmccabe/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/brianmccabe/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/brianmccabe/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [81]:
from tqdm import tqdm
tqdm.pandas()

In [82]:
# we can define a function that removes stopwords 
def process_tweet(tweet):
    tweet = str(tweet).lower()
    tokens = nltk.word_tokenize(tweet)
    stopwords_removed = [token.lower() for token in tokens if token.lower() not in stopwords]
    return stopwords_removed

In [83]:
#set stopwords and punctuations
stopwords = stopwords.words('english')
stopwords += list(string.punctuation)
stopwords += ["n't", "' '", "'re'","”","``","“","''","’","'s","'re","http","https", "rt"]
alph = ['a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z']
stopwords += alph

In [84]:
stopwords = list(set(stopwords))

In [85]:
def remove_http(tweet):
    pattern = '((http|https)\w+\s\w+\s\w+\s\w+)'
    try:
        return tweet.replace(re.findall(pattern, tweet)[0][0], "")
    except:
        return tweet

In [86]:
def capital_percentage(tweet):
    tokens = nltk.word_tokenize(tweet)
    cap_count = 0
    for item in tokens:
        if item.isupper():
            cap_count += 1
    return cap_count/len(tokens)

In [87]:
def check_profanity(tweet):
    profane = pd.read_csv("profane_words.csv", header=None)

    profane = list(profane.loc[:,0])
    count = 0
    tweet = tweet.lower()
    tokens = nltk.word_tokenize(tweet)
    for word in tokens:
        if word in profane:
            count += 1
    return count/len(tweet)

In [88]:
def fix_spelling(tweet):
    b = TextBlob(tweet)
    return b.correct()

In [89]:
def get_subjectivity(tweet):
    b = TextBlob(tweet)
    return b.sentiment.subjectivity

In [90]:
from nltk.stem import WordNetLemmatizer, PorterStemmer
  
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer()

In [91]:
def remove_username(tweet):
    try:
        p = '[\w\s]+(@\w+)'
        return tweet.replace(re.findall(p, tweet)[0], "")
    except:
        return tweet

In [92]:
def ReplaceThreeOrMore(tweet):
    # pattern to look for three or more repetitions of any character, including
    # newlines.
    pattern = re.compile(r"(.)\1{2,}", re.DOTALL) 
    return pattern.sub(r"\1\1", tweet)

In [93]:
def clean_txt(tweet):
    tweet = remove_http(tweet)
    tweet = remove_username(tweet)
    tweet = ReplaceThreeOrMore(tweet)
    tokens = process_tweet(tweet)
    return ' '.join([lemmatizer.lemmatize(w) for w in tokens])
    

In [94]:
analyzer = SentimentIntensityAnalyzer()
def return_sentiment(tweet):
    return analyzer.polarity_scores(tweet)['compound']

# 2. Load in Data and Preprocess

In [95]:
df = pd.read_csv("tweets_4_classes.csv").drop(['Unnamed: 0', 'emoji_frequency'], axis=1)
df.head()

,tweet,sentiment_score,top_emoji
0,I ll kidnap 1000 children before I let this co...,-0.2942,😊
1,AméricaniseUnTitre The Trump Tower Infernale,0.4588,😊
2,HappyBirthdayKendallJenner now go vote for bid...,0.6705,😊
3,Nashies y all know who to vote for @CharlesEst...,0.7424,😊
4,NoWayJoe You just tell lies Make promises Make...,0.4125,😭


In [96]:
print(df.tweet.iloc[0])
print(clean_txt(df.tweet.iloc[0]))
print(type(clean_txt(df.tweet.iloc[0])))

I ll kidnap 1000 children before I let this company die Mr Waternoose And probably Joe Biden
kidnap 100 child let company die mr waternoose probably joe biden
<class 'str'>


In [97]:
# # Remove "http link stuff from all the tweets"
# print(df.tweet.iloc[0])
# print(remove_http(df.tweet.iloc[0]))

# df.tweet = df.tweet.apply(remove_http)

In [98]:
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
normalizer = MinMaxScaler()
df.sentiment_score = normalizer.fit_transform(np.array(df.sentiment_score).reshape(-1,1))
df.sentiment_score.describe()

count    2507.000000
mean        0.486203
std         0.325251
min         0.000000
25%         0.176378
50%         0.436967
75%         0.826383
max         1.000000
Name: sentiment_score, dtype: float64

In [99]:
df['capitalization'] = df.tweet.progress_apply(capital_percentage)
df.head()

100%|██████████| 2507/2507 [00:00<00:00, 8795.97it/s]


,tweet,sentiment_score,top_emoji,capitalization
0,I ll kidnap 1000 children before I let this co...,0.353558,😊,0.117647
1,AméricaniseUnTitre The Trump Tower Infernale,0.731456,😊,0.000000
2,HappyBirthdayKendallJenner now go vote for bid...,0.837699,😊,0.000000
3,Nashies y all know who to vote for @CharlesEst...,0.873783,😊,0.000000
4,NoWayJoe You just tell lies Make promises Make...,0.708220,😭,0.000000


In [100]:
df['profanity'] = df.tweet.progress_apply(check_profanity)
df.head()

100%|██████████| 2507/2507 [00:05<00:00, 490.74it/s]


,tweet,sentiment_score,top_emoji,capitalization,profanity
0,I ll kidnap 1000 children before I let this co...,0.353558,😊,0.117647,0.01087
1,AméricaniseUnTitre The Trump Tower Infernale,0.731456,😊,0.000000,0.00000
2,HappyBirthdayKendallJenner now go vote for bid...,0.837699,😊,0.000000,0.00000
3,Nashies y all know who to vote for @CharlesEst...,0.873783,😊,0.000000,0.00000
4,NoWayJoe You just tell lies Make promises Make...,0.708220,😭,0.000000,0.00800


In [101]:
df['subjectivity'] = df.tweet.progress_apply(get_subjectivity)

100%|██████████| 2507/2507 [00:00<00:00, 5514.17it/s]


In [102]:
# test the replacing of extra chars
test = "yoooooo let's gooooo to the zoooo. Wazzzzuppppp. AAABBBCCC"

In [103]:
ReplaceThreeOrMore(test)

"yoo let's goo to the zoo. Wazzupp. AABBCC"

In [104]:
df.top_emoji.value_counts()

😭    1263
😊     947
😡     192
😱     105
Name: top_emoji, dtype: int64

# 3. Dummy Classifier for Baseline Model

In [105]:
X = df[['tweet', 'sentiment_score', 'capitalization', 'profanity']]

In [106]:
y =df['top_emoji']

In [107]:
from sklearn.dummy import DummyClassifier

In [108]:
dummy_cf = DummyClassifier(strategy='uniform')
dummy_cf.fit(X['tweet'],y)
y_preds = dummy_cf.predict(X['tweet'])

print(dummy_cf.score(X['tweet'],y))

0.2516952532907858


In [109]:
accuracy = accuracy_score(y, y_preds)


In [110]:
results=[]
results.append(('Dummy', accuracy))

In [111]:
results

[('Dummy', 0.2449142401276426)]

In [112]:
df.top_emoji.value_counts()

😭    1263
😊     947
😡     192
😱     105
Name: top_emoji, dtype: int64

### RESAMPLE

In [113]:
from sklearn.utils import resample
cry = df[df.top_emoji == '😭']
happy = df[df.top_emoji == '😊']
fear = df[df.top_emoji == '😱']
anger = df[df.top_emoji == '😡']


cry_downsampled = resample(cry,
                          replace=False,
                          n_samples=int(len(fear)*1.5), # match number
                          random_state=seed) 

happy_downsampled = resample(happy,
                          replace=False,
                          n_samples=int(len(fear)*1.5), # match number 
                          random_state=seed) 
fear_upsampled = resample(fear,
                          replace=True, 
                          n_samples=int(len(fear)*1.5), # match number 
                          random_state=seed) 
anger_upsampled = resample(anger,
                          replace=True,
                          n_samples=int(len(fear)*1.5), # match number 
                          random_state=seed) 

df = pd.concat([cry_downsampled, happy_downsampled, fear_upsampled, anger_upsampled])
df.top_emoji.value_counts()

😡    157
😱    157
😭    157
😊    157
Name: top_emoji, dtype: int64

In [114]:
X = df[['tweet', 'sentiment_score', 'capitalization', 'profanity']]

In [115]:
y =df['top_emoji']

# 4. Supervised Learning

In [116]:
import en_core_web_sm
nlp = en_core_web_sm.load()
import spacy 
from sklearn.base import BaseEstimator, TransformerMixin
class SpacyVectorTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, nlp):
        self.nlp = nlp
        self.dim = 300

    def fit(self, X, y):
        return self

    def transform(self, X):
        # Doc.vector defaults to an average of the token vectors.
        # https://spacy.io/api/doc#vector
        
        return [self.nlp(text).vector for text in X]

In [117]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import FeatureUnion, Pipeline
from sklearn.feature_extraction import DictVectorizer
class ItemSelector(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key

    def fit(self, x, y=None):
        return self

    def transform(self, data_dict):
        return data_dict[self.key]


class TextStats(BaseEstimator, TransformerMixin):
    """Extract features from each document for DictVectorizer"""

    def fit(self, x, y=None):
        return self

    def transform(self, data):
        return [{'cap':  row['capitalization'], 'prof': row['profanity'], 'sent': row['sentiment_score']} for _, row in data.iterrows()]
    

In [118]:
pipeline = Pipeline([
    ('union', FeatureUnion(
        transformer_list=[

            # Pipeline for pulling features from the text
            ('text', Pipeline([
                ('selector', ItemSelector(key='tweet')),
                ('tfidf', TfidfVectorizer( min_df =3, max_df=0.2, max_features=None, 
                    strip_accents='unicode', analyzer='word',token_pattern=r'\w{1,}',
                    ngram_range=(1, 2), use_idf=1,smooth_idf=1,sublinear_tf=1,
                    stop_words = None, preprocessor=clean_txt)),
            ])),
            
#             ('embedding', Pipeline([
#                 ('selector', ItemSelector(key='tweet')),
#                 ("mean_embeddings", SpacyVectorTransformer(nlp))
#             ])),

            # Pipeline for pulling metadata features
            ('stats', Pipeline([
                ('selector', ItemSelector(key=['capitalization', 'profanity', 'sentiment_score'])),
                ('stats', TextStats()),  # returns a list of dicts
                ('vect', DictVectorizer()),  # list of dicts -> feature matrix
            ])),

        ],

        # weight components in FeatureUnion
        transformer_weights={
            'text': 1,#0.9,
            #'embedding': 1,
            'stats': 1 #1.5,
        },
    ))
], verbose=True)

In [119]:
seed = 42
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=seed, stratify=y)

In [120]:
pipeline.fit(X_train)

[Pipeline] ............. (step 1 of 1) Processing union, total=   0.2s


Pipeline(steps=[('union',
                 FeatureUnion(transformer_list=[('text',
                                                 Pipeline(steps=[('selector',
                                                                  ItemSelector(key='tweet')),
                                                                 ('tfidf',
                                                                  TfidfVectorizer(max_df=0.2,
                                                                                  min_df=3,
                                                                                  ngram_range=(1,
                                                                                               2),
                                                                                  preprocessor=<function clean_txt at 0x126324af0>,
                                                                                  smooth_idf=1,
                                                           

In [121]:
%%time
train_vec = pipeline.transform(X_train)
test_vec = pipeline.transform(X_test)
print("Checking that the shapes match: %s - %s" % (train_vec.shape, test_vec.shape))

Checking that the shapes match: (502, 612) - (126, 612)
CPU times: user 227 ms, sys: 2.77 ms, total: 230 ms
Wall time: 229 ms


### LogReg

Algorithm to use in the optimization problem.

    - For small datasets, 'liblinear' is a good choice, whereas 'sag' and
      'saga' are faster for large ones.
    - For multiclass problems, only 'newton-cg', 'sag', 'saga' and 'lbfgs'
      handle multinomial loss; 'liblinear' is limited to one-versus-rest
      schemes.
    - 'newton-cg', 'lbfgs' and 'sag' only handle L2 penalty, whereas
      'liblinear' and 'saga' handle L1 penalty.
    - 'liblinear' might be slower in LogisticRegressionCV because it does
      not handle warm-starting.


In [122]:
from sklearn.linear_model import LogisticRegressionCV, LogisticRegression

In [123]:
lr_clf = LogisticRegressionCV(solver='newton-cg', cv=10, penalty='l2', Cs = [.001,.01,.1,1,10,100], 
                                    max_iter=10000, verbose=True, n_jobs=-1, scoring='f1', multi_class='ovr',
                                class_weight='balanced')
lr_clf.fit(train_vec, y_train)
test_preds = lr_clf.predict(test_vec)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:    0.9s finished


In [124]:
accuracy = accuracy_score(y_test, test_preds)
print('LogReg')
print("Testing Accuracy: {:.4}".format(accuracy))

results.append(('LogReg', accuracy))

LogReg
Testing Accuracy: 0.7857


In [125]:
print(classification_report(y_test, test_preds))
print('----------------------------------------')
print(confusion_matrix(y_test, test_preds))

              precision    recall  f1-score   support

           😊       0.82      0.97      0.89        32
           😡       0.88      0.68      0.76        31
           😭       0.79      0.59      0.68        32
           😱       0.70      0.90      0.79        31

    accuracy                           0.79       126
   macro avg       0.80      0.79      0.78       126
weighted avg       0.80      0.79      0.78       126

----------------------------------------
[[31  1  0  0]
 [ 4 21  2  4]
 [ 3  2 19  8]
 [ 0  0  3 28]]


### SVM(s)

#### LinearSVC

In [126]:
from sklearn.svm import LinearSVC

In [127]:
#Linear Support Vector Machines
sv_clf = LinearSVC(C=1, class_weight='balanced', multi_class='ovr', random_state=seed,verbose=3) 
sv_clf.fit(train_vec, y_train)
test_preds = sv_clf.predict(test_vec)

[LibLinear]

In [128]:
accuracy = accuracy_score(y_test, test_preds)
print('Linear SVC')
print("Testing Accuracy: {:.4}".format(accuracy))

results.append(('Linear SVC', accuracy))

Linear SVC
Testing Accuracy: 0.7143


#### GridSVC

In [129]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

In [130]:
param_grid = {'C':[.1,1,10,100],'gamma':[100,10,1,0.1,0.001], 'kernel':['linear','rbf']}

In [131]:
grid_svc=GridSearchCV(SVC(), param_grid, cv=10, scoring='accuracy', n_jobs=-1, verbose=1)
grid_svc.fit(train_vec, y_train)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


Fitting 10 folds for each of 40 candidates, totalling 400 fits


[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed:    0.5s
[Parallel(n_jobs=-1)]: Done 400 out of 400 | elapsed:    1.9s finished


GridSearchCV(cv=10, estimator=SVC(), n_jobs=-1,
             param_grid={'C': [0.1, 1, 10, 100],
                         'gamma': [100, 10, 1, 0.1, 0.001],
                         'kernel': ['linear', 'rbf']},
             scoring='accuracy', verbose=1)

In [132]:
print(grid_svc.best_score_)
print(grid_svc.best_params_)
print(grid_svc.best_estimator_)

0.7770588235294118
{'C': 1, 'gamma': 1, 'kernel': 'rbf'}
SVC(C=1, gamma=1)


In [133]:
test_preds = grid_svc.best_estimator_.predict(test_vec)

In [134]:
accuracy = accuracy_score(y_test, test_preds)
print('SVC Grid')
print("Testing Accuracy: {:.4}".format(accuracy))

SVC Grid
Testing Accuracy: 0.7143


In [135]:
results.append(('SVC_Grid', accuracy))
results

[('Dummy', 0.2449142401276426),
 ('LogReg', 0.7857142857142857),
 ('Linear SVC', 0.7142857142857143),
 ('SVC_Grid', 0.7142857142857143)]

### Random Forest

In [136]:
rfc_clf = RandomForestClassifier(n_estimators=250, random_state=seed,n_jobs=-1,verbose=1, class_weight='balanced')
rfc_clf.fit(train_vec, y_train)
test_preds = rfc_clf.predict(test_vec)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 250 out of 250 | elapsed:    0.3s finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 250 out of 250 | elapsed:    0.0s finished


In [137]:
accuracy = accuracy_score(y_test, test_preds)
print('Random Forest')
print("Testing Accuracy: {:.4}".format(accuracy))

results.append(('RFC', accuracy))

Random Forest
Testing Accuracy: 0.7302


In [138]:
print(classification_report(y_test, test_preds))
print('----------------------------------------')
print(confusion_matrix(y_test, test_preds))

              precision    recall  f1-score   support

           😊       0.75      0.94      0.83        32
           😡       0.72      0.68      0.70        31
           😭       0.64      0.56      0.60        32
           😱       0.79      0.74      0.77        31

    accuracy                           0.73       126
   macro avg       0.73      0.73      0.72       126
weighted avg       0.73      0.73      0.72       126

----------------------------------------
[[30  0  1  1]
 [ 6 21  4  0]
 [ 3  6 18  5]
 [ 1  2  5 23]]


### MN Bayes

In [139]:
#Multinomial Naive Bayes
mnb_clf = MultinomialNB() 
mnb_clf.fit(train_vec, y_train)
test_preds = mnb_clf.predict(test_vec)

In [140]:
accuracy = accuracy_score(y_test, test_preds)
print('MN Bayes')
print("Testing Accuracy: {:.4}".format(accuracy))

results.append(('MNBayes', accuracy))

MN Bayes
Testing Accuracy: 0.5556


### Bernoulli Bayes

In [141]:
#Bernoulli Naive Bayes
bb_clf = BernoulliNB() 
bb_clf.fit(train_vec, y_train)
test_preds = bb_clf.predict(test_vec)

In [142]:
accuracy = accuracy_score(y_test, test_preds)
print('Bernoulli Bayes')
print("Testing Accuracy: {:.4}".format(accuracy))

results.append(('BerBayes', accuracy))

Bernoulli Bayes
Testing Accuracy: 0.4444


In [143]:
results

[('Dummy', 0.2449142401276426),
 ('LogReg', 0.7857142857142857),
 ('Linear SVC', 0.7142857142857143),
 ('SVC_Grid', 0.7142857142857143),
 ('RFC', 0.7301587301587301),
 ('MNBayes', 0.5555555555555556),
 ('BerBayes', 0.4444444444444444)]

### Passive Aggressive Classifier

In [144]:
from sklearn.linear_model import PassiveAggressiveClassifier

In [145]:
#PassiveAggresive Classifier
pac_clf = PassiveAggressiveClassifier() 
pac_clf.fit(train_vec, y_train)
test_preds = pac_clf.predict(test_vec)

In [146]:
accuracy = accuracy_score(y_test, test_preds)
print('MN Bayes')
print("Testing Accuracy: {:.4}".format(accuracy))

results.append(('PassiveAgg', accuracy))

MN Bayes
Testing Accuracy: 0.7143


In [147]:
results

[('Dummy', 0.2449142401276426),
 ('LogReg', 0.7857142857142857),
 ('Linear SVC', 0.7142857142857143),
 ('SVC_Grid', 0.7142857142857143),
 ('RFC', 0.7301587301587301),
 ('MNBayes', 0.5555555555555556),
 ('BerBayes', 0.4444444444444444),
 ('PassiveAgg', 0.7142857142857143)]

### XGBoost

In [148]:
import xgboost as xgb

In [149]:
# RUN A BASELINE XGB
xg = xgb.XGBClassifier()
xg.fit(train_vec, y_train)
test_preds = xg.predict(test_vec)

In [150]:
accuracy = accuracy_score(y_test, test_preds)
print('XGBoost')
print("Testing Accuracy: {:.4}".format(accuracy))

results.append(('XGB', accuracy))

XGBoost
Testing Accuracy: 0.746


In [151]:
results

[('Dummy', 0.2449142401276426),
 ('LogReg', 0.7857142857142857),
 ('Linear SVC', 0.7142857142857143),
 ('SVC_Grid', 0.7142857142857143),
 ('RFC', 0.7301587301587301),
 ('MNBayes', 0.5555555555555556),
 ('BerBayes', 0.4444444444444444),
 ('PassiveAgg', 0.7142857142857143),
 ('XGB', 0.746031746031746)]

In [152]:
from sklearn.metrics import f1_score

In [153]:
print(classification_report(y_test, test_preds))
print('---------------------------------------')
print(confusion_matrix(y_test, test_preds))

              precision    recall  f1-score   support

           😊       0.81      0.94      0.87        32
           😡       0.63      0.71      0.67        31
           😭       0.68      0.53      0.60        32
           😱       0.86      0.81      0.83        31

    accuracy                           0.75       126
   macro avg       0.75      0.75      0.74       126
weighted avg       0.75      0.75      0.74       126

---------------------------------------
[[30  2  0  0]
 [ 5 22  4  0]
 [ 2  9 17  4]
 [ 0  2  4 25]]


### Voting Classifier

In [154]:
from sklearn.ensemble import VotingClassifier

In [155]:
voting_clf = VotingClassifier(
                estimators=[('logreg', lr_clf), ('svm_linear', sv_clf), ('pass_aggr', pac_clf),
                            ('smv_grid', grid_svc.best_estimator_), ('xgboost', xg), ('rfc', rfc_clf),
                            ('mnbayes', mnb_clf),('berbayes', bb_clf)], #
                voting='hard', verbose=1, n_jobs= -1)
voting_clf.fit(train_vec, y_train)
test_preds = voting_clf.predict(test_vec)

[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 250 out of 250 | elapsed:    0.0s finished


In [156]:
accuracy = accuracy_score(y_test, test_preds)
print('Voting')
print("Testing Accuracy: {:.4}".format(accuracy))

results.append(('Voting', accuracy))

Voting
Testing Accuracy: 0.6984


In [157]:
results = sorted(results, key=lambda x: x[1], reverse=True)
results

[('LogReg', 0.7857142857142857),
 ('XGB', 0.746031746031746),
 ('RFC', 0.7301587301587301),
 ('Linear SVC', 0.7142857142857143),
 ('SVC_Grid', 0.7142857142857143),
 ('PassiveAgg', 0.7142857142857143),
 ('Voting', 0.6984126984126984),
 ('MNBayes', 0.5555555555555556),
 ('BerBayes', 0.4444444444444444),
 ('Dummy', 0.2449142401276426)]

### Bar Chart of Different Model Results

In [ ]:
x = [x[0] for x in results]
y = [x[1] for x in results]

In [ ]:
plt.figure(figsize=(12,6))
sns.barplot(x,y)
plt.ylabel('Accuracy')
plt.title("Early Results", fontsize=20)
#plt.savefig("../pics/model_performances.png")